In [1]:
from resume_parser import resumeparse
from tika import parser  
import locationtagger
import re
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
from pdfminer.high_level import extract_text
import aspose.words as aw
import pandas as pd
import docx2txt
from PyPDF2 import PdfReader
import os
import pathlib
####################################################

import glob
from pdfminer.high_level import extract_text
import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_lg")
import docx


/home/anush/anaconda3/lib/python3.9/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
base_dir = pathlib.Path(__name__).parent.absolute()
pdf_folder_path = os.path.join(base_dir,'new_pdf',)
pdf_file_path  = os.path.join(base_dir,'new_pdf','*')
pdf_folder_path

'/home/anush/Desktop/flask_name/new_pdf'

In [3]:
def doc_convert_to_pdf(path):
#     count =0
    cunt =0
    cnt = 0
    for file_ in glob.glob(path):
        try:
            if file_.endswith(".doc"):
                print(file_,'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa')
                doc = aw.Document(file_)
                doc.save(os.path.join(base_dir,'Training_resume',f"+{cunt}.pdf"))
                cunt+=1
            elif file_.endswith(".rtf"):
                print(file_,'bbbbbbbbbbbbbbbbbbbbbbbbbbbbbb')
                doc = aw.Document(file_)
                doc.save(os.path.join(base_dir,'Training_resume',f"++{cunt}.pdf"))
                cnt+=1        
        except  Exception as e:
            print(e)


#convert word to text
def  extract_text_from_word(word_path):
    doc = docx.Document(word_path)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)


#convert pdf to text
def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)


#return text
def return_text(path):
    if path.endswith('.docx'):
        return extract_text_from_word(path)
    elif path.endswith('.pdf'):
        return extract_text_from_pdf(path)
    

#convert_the_info_dic
def convert_the_info_dic(text):
    #breaking the text
    dit ={}
    title = ['Education','Qualification','Skills','Objective','Experience','Tools','Designation','Employment']
    for i in title:
        text = text.replace(f'{i}',f'|{i}').replace(f'{i.upper()}',f'|{i.upper()}')
    text = text.split("|")

    #making dictionary of inforamtion
    for titl in title:
        check = False
        for txt in text: 
            if titl in txt or titl.upper() in txt:
                check = True
                dit[titl] = txt 
        if check == False:
            dit[titl] = 'Nan'
    return dit


#extraction_the_data
def extraction_of_Data(resume_path,data_in_dic,text):
    prev =[]
    person =[]
    city = []
    education = []
    skill =[]
    doc = nlp(text)


#     extraction of Education
    edu_attribute = data_in_dic['Education']
    education.append(edu_attribute)
    
    data1 = resumeparse.read_file(resume_path)
    #extract the name
    name_data = data1['name']
    if 'Aspose Pty' == name_data:
        people =[]
        for ent in doc.ents:
            if ent.label_ == "PERSON" or (ent.text.istitle()):
                people.append(ent.text)
        name_data = people[2]
    elif 'curriculum vitae' == name_data.lower():
        people =[]
        for ent in doc.ents:
            if ent.label_ == "PERSON" or (ent.text.istitle()):
                people.append(ent.text)
                break
        name_data = people[0]
    else:
        name_data = name_data
    #extract the email
    email_data = data1['email']
    #extract the phone
    phone = data1['phone']
    if str(phone)=='003-2023':
        phone_pattern = re.compile(r'(?:\+\d{1,2}\s)?\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}')
        phone_numbers = re.findall(phone_pattern, text)
        phone =phone_numbers[0]
    else:
        phone = phone

    #extract the designition
    designation_data = data1['designition']
    
    #extract the experience
    experience_data = data1['total_exp']
    if len([i for i in str(experience_data)])>2:
        experience_data = None
    else:
        experience_data = experience_data
        
    #extract the skills
#     skill_data = data1['skills']
        
    #extract the New_skills    
    skill_attribute = data_in_dic['Skills']
    if str(skill_attribute) =='Nan':
        skill.append(data1['skills'])
    else:
        skill_attribute = data_in_dic['Skills']
        skill.append(skill_attribute)
    
    #append the information 
#     prev.append(person)
    prev.append(name_data)
    prev.append(email_data)
    prev.append(phone)
    prev.append(skill)
    prev.append(designation_data)
    prev.append(experience_data)
    prev.append(education)
#     prev.append(city)
#location
    place_entity = locationtagger.find_locations(text = text)
    prev.append(str(place_entity.countries))
    prev.append(str(place_entity.regions))
    prev.append(str(place_entity.cities))
    prev.append(resume_path)
    return prev 

In [4]:
df = pd.DataFrame(columns=['Name', 'Email','Mobile','Skills','Designation','Experience_Period','education','countries','regions','cities','Resume_path'])

In [ ]:

doc_convert_to_pdf(pdf_file_path)

for path_of_list in glob.glob(pdf_file_path):
    
    if path_of_list.endswith(".pdf"):
        try:
            text = return_text(path_of_list)
            data_in_dic = convert_the_info_dic(text)
            data = extraction_of_Data(path_of_list,data_in_dic, text)
    ########################creating the dataframe###############################
            df.loc[-1] = data
            df.reset_index(drop ='index',inplace =True)
        except Exception as e:
            print(e)
    elif path_of_list.endswith(".docx"):
        try:
            text = return_text(path_of_list)
            data_in_dic = convert_the_info_dic(text)
            data = extraction_of_Data(path_of_list,data_in_dic, text)
    ########################creating the dataframe###############################
            df.loc[-1] = data
            df.reset_index(drop ='index',inplace =True)
        except Exception as e:
            print(e)
        

list index out of range


ERROR:root:'NoneType' object has no attribute 'group'
ERROR:root:invalid literal for int() with base 10: 'October2010'
ERROR:root:'NoneType' object has no attribute 'group'
ERROR:root:invalid literal for int() with base 10: 'feb15'
ERROR:root:'NoneType' object has no attribute 'group'
ERROR:root:'NoneType' object has no attribute 'group'


Please input any text or url


ERROR:root:invalid literal for int() with base 10: '2012-14'
ERROR:root:invalid literal for int() with base 10: 'July2015'
ERROR:root:'NoneType' object has no attribute 'group'
ERROR:root:invalid literal for int() with base 10: 'till Date'
ERROR:root:'NoneType' object has no attribute 'group'
ERROR:root:invalid literal for int() with base 10: 'july2018'
ERROR:root:invalid literal for int() with base 10: 'date'
ERROR:root:'NoneType' object has no attribute 'group'
ERROR:root:invalid literal for int() with base 10: 'March2014'
ERROR:root:invalid literal for int() with base 10: '10:19'
ERROR:root:'NoneType' object has no attribute 'group'


Please input any text or url


ERROR:root:'NoneType' object has no attribute 'group'
ERROR:root:'NoneType' object has no attribute 'group'
ERROR:root:invalid literal for int() with base 10: '25.10.14'
ERROR:root:invalid literal for int() with base 10: '03-july-2011'
ERROR:root:invalid literal for int() with base 10: 'july2018'
ERROR:root:invalid literal for int() with base 10: 'date'
ERROR:root:'NoneType' object has no attribute 'group'
ERROR:root:invalid literal for int() with base 10: 'Feb.2016'
ERROR:root:invalid literal for int() with base 10: '08 12'
ERROR:root:invalid literal for int() with base 10: 'Feb.2012'
ERROR:root:invalid literal for int() with base 10: 'Till date'
ERROR:root:'NoneType' object has no attribute 'group'
ERROR:root:invalid literal for int() with base 10: 'July 2013'
ERROR:root:invalid literal for int() with base 10: 'Jan-06'
ERROR:root:invalid literal for int() with base 10: '02-16'
ERROR:root:invalid literal for int() with base 10: '11-oct-2014'
ERROR:root:invalid literal for int() with b

Please input any text or url


In [ ]:
df 

In [ ]:
df.to_csv(os.path.join(base_dir,'data_csv','client.csv'))

In [ ]:
for i in df['Skills']:
    print(i,'gggggggggggggggggg')

# data-Cleaning

In [ ]:
# def cities(x):
#     x =x[1:-1]
#     doc = nlp(x)
#     citi=[]
#     for ent in doc.ents:
#         if ent.label_ == 'GPE'or ent.label_ =='NORP' or ent.label_ == 'FAC':
#             citi.append(ent.text)
#     return citi

# df['cities'] = df['cities'].apply(lambda x :cities(x))

In [ ]:
# df.to_csv('resume_extration_2_5000.csv', index = False)

In [ ]:
# pd.read_csv('resume_extration_2_5000.csv')

In [ ]:
##education_____________

# ad = r'Created\s+with\s+an\s+evaluation\s+copy\s+of\s+Aspose\.Words\.\s+To\s+discover\s+the\s+full\s+versions\s+of\s+our\s+APIs\s+please\s+visit:\s+https://products\.aspose\.com/words/'
# df['New_Skill'] =df['New_Skill'].apply(lambda x : re.sub(ad, "", x))
# a,b,c,d,e= '\\x0c','\\xa0','\\u200b','Education','EDUCATION'
# dat = [i.replace(a, '').replace(b,'').replace(c,'').replace(d,'').replace(e,'') for i in df['New_Skill'].values]
# df['New_Skill'] = dat
# df['New_Skill'].replace('\t',"")
# df['New_Skill'].replace('\n',"")
# df['New_Skill'].replace('\n\n',"")


In [ ]:
import re

df['Skills'].replace(to_replace=[r"\\t|\\n|\\r", r'\r+|\n+|\t+',r"\t|\n|\r"], value=["","",""], regex=True, inplace=True)
df.Skills = df.Skills.apply(lambda x: re.sub(r'@[\S]+', '', str(x)))
df.Skills = df.Skills.apply(lambda x: re.sub(r'&[\S]+?;', '', str(x)))
df.Skills = df.Skills.apply(lambda x: re.sub(r'#', ' ', str(x)))
df.Skills = df.Skills.apply(lambda x: re.sub(r'(\bRT\b|\bQT\b)', '', str(x)))
df.Skills = df.Skills.apply(lambda x: re.sub(r'http[\S]+', '', str(x)))
df.Skills = df.Skills.apply(lambda x: re.sub(r'\w*\d\w*', r'', str(x)))
df.Skills = df.Skills.apply(lambda x: re.sub(r'\s\s+', ' ', str(x)))
df.Skills = df.Skills.apply(lambda x: re.sub(r'(\A\s+|\s+\Z)', '', str(x)))
df['Skills'].replace('\t',"").replace('\n',"").replace('\n\n',"").replace("\\","")
df.Skills.head(10)

In [ ]:
import string
string.punctuation
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

df['New_Skill']= df['New_Skill'].apply(lambda x: remove_punct(x))

In [ ]:
def tokenization(text):
    text = re.split('\W+', str(text))
    return text

df.New_Skill = df['New_Skill'].apply(lambda x: tokenization(x.lower()))

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
wn = nltk.WordNetLemmatizer()


wn = nltk.WordNetLemmatizer()
def get_wordnet_pos(treebank_tag):
    treebank_tag =str(treebank_tag)
    if treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
def lemmatzer(text):
    words_and_tags = nltk.pos_tag(text)
    lem = []
    for word, tag in words_and_tags:
        lemma = wn.lemmatize(word,pos =get_wordnet_pos(tag))
        lem.append(lemma)
    return lem


df.New_Skill= df.New_Skill.apply(lambda x: lemmatzer(x))

In [ ]:
for i in df.New_Skill:
    print(i)